<div class="alert alert-block alert-info">
    <h1>A practial introduction to policy search in RL </h1><br>
    <span>Algerian AI Summer University, August 2021</span>
</div>

# 2. Implementing Policy Gradient

## Imports, configuration, and useful functions

First, installing some libraries if not done already in your terminal

In [ ]:
!pip install torch==1.9.0 torchvision pyvirtualdisplay gym numpy pandas python-box plotly tqdm
!sudo apt-get install xvfb

In [4]:
import numpy as np
import pandas as pd

In [5]:
import gym
from gym.wrappers import Monitor

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

A utility function to display short videos of the environments

In [7]:
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from pathlib import Path
import tempfile
import base64

def show_episode(env, policy):
    with tempfile.TemporaryDirectory() as tmpdir:
        logs_dir = tmpdir
        env = Monitor(env, logs_dir, force=True, video_callable=lambda episode: True)
        interact(env, policy)
        html = []
        for mp4 in Path(logs_dir).glob("*.mp4"):
            video_b64 = base64.b64encode(mp4.read_bytes())
            html.append('''<video alt="{}" autoplay 
                          loop controls style="height: 400px;">
                          <source src="data:video/mp4;base64,{}" type="video/mp4" />
                     </video>'''.format(mp4, video_b64.decode('ascii')))
        ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))
    env.close()

In [8]:
display = Display(visible=0, size=(1200, 800))
display.start();

Setting the random seeds to make the random outputs reproducible. Mandatory for debugging, developing algorithms, but proscribed when running exeperiments.

In [9]:
import numpy as np

`torch` imports

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as D
from torch import optim

Setting the random seeds to make the random outputs reproducible. Mandatory for debugging, developing algorithms, but proscribed when running exeperiments.

In [11]:
seed = 1337
np.random.seed(seed=seed)
torch.manual_seed(seed=seed);

Plotting imports

In [12]:
from tqdm.notebook import tqdm, trange

In [13]:
from box import Box

In [14]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## The env

In [15]:
env = gym.make("CartPole-v0")

In [16]:
env.seed(seed);

Keep in mind this is by default a "wrapped" environment, to limit the number of steps

In [17]:
env

<TimeLimit<CartPoleEnv<CartPole-v0>>>

In [18]:
env._max_episode_steps

200

To get the unwrapped env, simply called the `unwrapped` attribute

In [19]:
env.unwrapped

## The policy $\pi$

A policy is a function of the observation (or the state) 
$$
a = \theta(s)
$$
or 
$$
a \sim \theta(a|s)
$$
in the stochastic case. 

One way to learn the policy is to parametrize it with a neural network and then infer the weights of the network $\theta$ an optimization technique (gradient-descent for instance).

In [20]:
# def make_policy(input_dim, n_output)
input_dim = env.observation_space.shape[0]
hidden_dim = 256
output_dim = env.action_space.n

In [21]:
class DescretePolicy(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.base_network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Softmax(dim=0)
        )
    def forward(self, x):
        if isinstance(x, np.ndarray):
            x = torch.from_numpy(x)
        x = x.float()  # use .unsqueeze(dim=0) if you want to treat a batch of states
        x = self.base_network(x)
        dist = D.Categorical(probs=x)
        action = dist.sample().squeeze()
        return action, dist

In [22]:
policy = DescretePolicy(input_dim, hidden_dim, output_dim)

In [23]:
policy

DescretePolicy(
  (base_network): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=2, bias=True)
    (5): Softmax(dim=0)
  )
)

In [24]:
def load_model_parameters(model: nn.Module, path: str):
    model.load_state_dict(torch.load(path))
def save_model_parameters(model: nn.Module, path: str):
    torch.save(model.state_dict(), path)

and we need a training function

## Training functions

We will need to define a couple of functions

In [25]:
# Exercise 
# compute the returns in one pass
    
def compute_return(rewards, discount):
    discounted_return = [0]
    for reward in reversed(rewards): 
        discounted_return.append(reward + discount * discounted_return[-1])

    discounted_return = np.array(discounted_return)
    discounted_return = discounted_return[1:][::-1]
    return discounted_return

In [26]:
def interact(env, policy, horizon=None):
    rewards = []
    log_probs = []
    entropies = []
    t, done = 1, False
    obs = env.reset()
    while t != horizon and not done:
        action, dist = policy(obs)
        new_obs, reward, done, _ = env.step(action.item())
        
        rewards.append(reward)
        log_probs.append(dist.log_prob(action))
        entropies.append(dist.entropy())
        
        obs = new_obs
        t += 1
    env.close()
    return rewards, log_probs, entropies

In [27]:
show_episode(env, policy)  # showing a random policy

In [28]:
def optimize(env, policy, optimizer, discount, batch_size):

    returned = Box()
    returned.loss = []
    returned.reward = []
    returned.entropy = []

    optimizer.zero_grad()

    for _ in trange(batch_size, desc='batch', leave=False, disable=False):
        rewards, log_probs, entropies = interact(env, policy, 200)  # FIXME: hard-coded episode length
        discounted_return = compute_return(rewards, discount)
        undiscounted_return = np.sum(rewards)

        # we normalize the returns 
        eps = np.finfo(np.float32).eps.item()
        discounted_return = (discounted_return - discounted_return.mean()) / (discounted_return.std() + eps)

        loss = []
        for lp, r in zip(log_probs, discounted_return):
            loss.append(lp * r)

        loss = torch.stack(loss)
        entropies = torch.stack(entropies)
        
        if any(torch.isnan(loss)):
            breakpoint()
            
        loss = loss.sum()
        loss = -loss
        loss.backward()

        returned.loss.append(loss.item())
        returned.reward.append(undiscounted_return)
        returned.entropy.append(entropies.mean().item())



    optimizer.step()

    returned.loss = np.mean(returned.loss)
    returned.reward = np.mean(returned.reward)
    returned.entropy = np.mean(returned.entropy)

    return returned

In [29]:
def evaluate(env, policy, n_episodes=100):
    ret = Box()
    with torch.no_grad():
        rewards = [interact(env, policy)[0] for ep in range(n_episodes)]    
    rewards = [np.sum(r) for r in rewards]
    ret["reward_mean"] = np.mean(rewards)
    ret["reward_std"] = np.std(rewards)
    return ret

In [30]:
def init_plot():
    fig = make_subplots(rows=3, cols=1)
    fig.add_trace(
        go.Scatter(
            x=[],
            y=[],
            line=dict(color="rgb(0,100,80)"),
            mode="lines",
            name="reward",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=[],
            y=[],
            fill="toself",
            fillcolor="rgba(0,100,80,0.2)",
            line=dict(color="rgba(255,255,255,0)"),
            hoverinfo="skip",
            showlegend=False,
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(x=[], y=[], line=dict(color="#D55D71"), mode="lines", name="entropy"),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(x=[], y=[], line=dict(color="#9F5EDE"), mode="lines", name="loss"),
        row=3,
        col=1,
    )
    fig.update_layout(height=800)
    return go.FigureWidget(fig)

In [31]:
def update_plot(fig, data):
    rew_std_upper = data.reward_mean + data.reward_std
    rew_std_lower = data.reward_mean - data.reward_std

    with fig.batch_update():
        fig.data[0].x = data.epoch
        fig.data[0].y = data.reward_mean
        fig.data[1].x = pd.concat((data.epoch, data.epoch.iloc[::-1]))
        fig.data[1].y = pd.concat((rew_std_upper, rew_std_lower.iloc[::-1]))
        fig.data[2].x = data.epoch
        fig.data[2].y = data.entropy
        fig.data[3].x = data.epoch
        fig.data[3].y = data.loss

## Actual training

Some training parameters

In [37]:
n_epochs = 100
n_eval_ep = 10      # number of episodes used for the evaluation
batch_size = 50
discount = 0.99     # discount factor gamma
lr = 3e-4           # learning rate

In [38]:
optimizer = optim.Adam(policy.parameters(), lr=lr)

Let's evaluate the policy before training (ie., a random policy) to get a baseline

In [39]:
print("Average reward (100 runs): {reward_mean} ± {reward_std:.3f}".format(**evaluate(env, policy, n_eval_ep)))

Average reward (100 runs): 52.5 ± 24.965


In [ ]:
fig = init_plot()

# this will display the figure (thanks to `InteractiveShell.ast_node_interactivity = "all"`)
fig 

loss = []
reward = []

data = pd.DataFrame()
current_epoch = 0

In [ ]:
for i in trange(current_epoch, current_epoch + n_epochs, desc="epoch", disable=False):

    train_results = optimize(env, policy, optimizer, discount, batch_size)
    eval_results = evaluate(env, policy, n_eval_ep)

    info = Box()
    info.epoch = i
    info.update(train_results)
    info.update(eval_results)

    data = data.append(info, ignore_index=True)
    
    save_model_parameters(policy, "model_cart.pt")
    update_plot(fig, data)
    current_epoch += 1

In [ ]:
print("Average reward (100 runs): {reward_mean} ± {reward_std:.3f}".format(**evaluate(env, policy, n_eval_ep)))

In [ ]:
show_episode(env, policy)

## A continous-action env (homework)

In [34]:
env = gym.make("Pendulum-v0")
env.seed(seed);

In [35]:
env._max_episode_steps = 15

In [36]:
input_dim = env.observation_space.shape[0]
hidden_dim = 64
n_hidden = 2
output_dim = env.action_space.shape[0]  # NOTICE

In [38]:
class ContinuousPolicy(nn.Module):
    def __init__(self, input_dim, n_hidden, hidden_dim, output_dim):
        super().__init__()
        self.base_network = nn.Sequential(
            *[nn.Linear(input_dim, hidden_dim), nn.ReLU()] * n_hidden,
            nn.Linear(hidden_dim, hidden_dim),
        )

        self.layer_mean = nn.Linear(hidden_dim, output_dim)
        self.layer_std_ = nn.Linear(hidden_dim, output_dim)
        self.layer_std = nn.Softplus()

    def forward(self, x):
        if isinstance(x, np.ndarray):
            x = torch.from_numpy(x)
        x = x.float()  # use .unsqueeze(dim=0) if you want to treat a batch of states
        x = self.base_network(x)

        mean = self.layer_mean(x)
        std_ = self.layer_std_(x)
        std = self.layer_std(std_)
        dist = D.Normal(mean, std)
        action = dist.sample().squeeze()

        return action, dist

In [40]:
policy = ContinuousPolicy(input_dim, n_hidden, hidden_dim, output_dim)
policy

ContinuousPolicy(
  (base_network): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
  )
  (layer_mean): Linear(in_features=64, out_features=1, bias=True)
  (layer_std_): Linear(in_features=64, out_features=1, bias=True)
  (layer_std): Softplus(beta=1, threshold=20)
)

In [145]:
n_epochs = 15000
n_eval_ep = 10      # number of episodes used for the evaluation
batch_size = 50
discount = 0.99     # discount factor gamma
lr = 3e-4           # learning rate

In [146]:
optimizer = optim.Adam(policy.parameters(), lr=lr)

In [147]:
def interact(env, policy, horizon=None):
    rewards = []
    log_probs = []
    entropies = []
    t, done = 1, False
    obs = env.reset()
    while t != horizon and not done:
#         obs_tensor = torch.from_numpy(obs).float()
        action, dist = policy(obs)
        new_obs, reward, done, _ = env.step([action.item()])
        
        rewards.append(reward)
        log_probs.append(dist.log_prob(action))
        entropies.append(dist.entropy())
        
        obs = new_obs
        t += 1
    env.close()
    return rewards, log_probs, entropies

In [148]:
print("Average reward (100 runs): {reward_mean} ± {reward_std:.3f}".format(**evaluate(env, policy, n_eval_ep)))

Average reward (100 runs): -109.36355033998525 ± 20.157


In [ ]:
fig = init_plot()

# this will display the figure (thanks to `InteractiveShell.ast_node_interactivity = "all"`)
fig 

loss = []
reward = []

data = pd.DataFrame()
current_epoch = 0

In [ ]:
for i in trange(current_epoch, current_epoch + n_epochs, desc="epoch", disable=False):

    train_results = optimize(env, policy, optimizer, discount, batch_size)
    eval_results = evaluate(env, policy, n_eval_ep)

    info = Box()
    info.epoch = i
    info.update(train_results)
    info.update(eval_results)

    data = data.append(info, ignore_index=True)
    
    save_model_parameters(policy, "model_cart.pt")
    update_plot(fig, data)
    current_epoch += 1

In [ ]:
print("Average reward (100 runs): {reward_mean} ± {reward_std:.3f}".format(**evaluate(env, policy, n_eval_ep)))

### Does it work? Probably not. Then try to implement an actor-critic algorithm.